In [2]:
import random
import numpy as np
import re

In [3]:
# preprocess text files to remove page markers, chapter markers and newlines.
for i in range(1,6):
    with open('GoT_txts/00{}ssb.txt'.format(i), 'r') as fin, \
            open('cleaned_GoT_txts/cleaned_00{}ssb.txt'.format(i), 'w') as fout:
        print("Cleaning {}".format(i))
        for line in fin:
#             fout.write(re.sub("Page \d+|^[A-Z ]{2,}", "", line) + " ")
            fout.write(re.sub("Page \d+","", line) + " ")
    fin.close()
    fout.close()
    
print("Cleaning Complete!")

Cleaning 1
Cleaning 2
Cleaning 3
Cleaning 4
Cleaning 5
Cleaning Complete!


In [4]:
# split into character files
for i in range(1,6):
    character_files = {}
    current_char = ""
    with open('cleaned_GoT_txts/cleaned_00{}ssb.txt'.format(i), 'r') as fin:
        for line in fin:
            line = line.rstrip()
            res = re.search("^[A-Z ]{3,}$", line)
            if res is not None:
                current_char = res.string
                if current_char not in character_files:
                    character_files[current_char] = open('character_txts/{}.txt'.format(current_char), 'w')
            elif current_char:
                character_files[current_char].write(line + " ")
    fin.close()
for c in character_files:
    character_files[c].close()
    
print("Character separation complete!")

Character separation complete!


In [5]:
def fill_empty(last_word, word, rmc):
    if last_word not in rmc:
        rmc[last_word] = {}
    if word not in rmc[last_word]:
        rmc[last_word][word] = 0


# Method 2

In [6]:
# Basic process word
def process_word(last_word, word, mc):
    fill_empty(last_word, word, mc)
    mc[last_word][word] += 1
    return word

In [7]:
# generate markov chain w/ punctuation as a state
# characters = ["ARYA.txt", "BRAN.txt", "JON.txt", "TYRION.txt"]
characters = ["EDDARD.txt"]
mc = {}
last_word = "."
for c in characters:
    with open('working_characters/{}'.format(c), "r") as fin:
        for line in fin:
            line = re.split('([?,.!"])',line)
            new_line = []
            for word in line:
                new_line += word.split()
            for word in new_line:
                last_word = process_word(last_word, word.lower(), mc)

In [8]:
# generate sentence

# Convert to probabilities
for word in mc:
    total = sum(mc[word].values())
    for next_word in mc[word]:
        mc[word][next_word] /= total
last_word = "."
end = False

sentence = ''
for _ in range(100):
    next_word = np.random.choice(list(mc[last_word].keys()), 1, list(mc[last_word].values()))[0]
    last_word = next_word
    sentence += last_word + " "
    if last_word in "?.!":
        end = True
print(sentence)

sons were gone with tears again at a dull ache in these things will have mercy . clegane's first grey , knights had given robert intend to punish me truly heal , written on tregar's helm for you don't recall that her father in kind . pick it , whose mother ? ned loved nothing in tricks , away to discuss , pink stone breaking . besides . don't just what right leg still holding tight little interest , hilt , far ? they still on either end or must have done everything that pretty mouth grew . eyes burning 


# Method 3

In [9]:
# Processes dialogue 
def process_word_diag(last_word, word, mc, dmc, diag=False):
    if word == '"':
        if not diag:
            fill_empty(last_word, "diag", mc)
            mc[last_word]["diag"] += 1
            return last_word, True
        if diag:
            fill_empty(last_word, "exitdiag", dmc)
            dmc[last_word]["exitdiag"] += 1
            return last_word, False
    if diag:
        rmc = dmc
    else:
        rmc = mc
    fill_empty(last_word, word, rmc)
    rmc[last_word][word] += 1
    return word, diag

In [10]:
mc = {}
dmc = {}
diag = False
last_word = "."
for c in characters:
    with open('city_and_sea.txt', "r") as fin:
        for line in fin:
            line = re.split('([?,.!"])',line)
            new_line = []
            for word in line:
                new_line += word.split()
            for word in new_line:
                last_word, diag = process_word_diag(last_word, word.lower(), mc, dmc, diag)

In [11]:
# generate markov chain w/ separate dialogue MC
# characters = ["ARYA.txt", "BRAN.txt", "JON.txt", "TYRION.txt"]
characters = ["EDDARD.txt"]
mc = {}
dmc = {}
diag = False
last_word = "."
for c in characters:
    with open('working_characters/{}'.format(c), "r") as fin:
        for line in fin:
            line = re.split('([?,.!"])',line)
            new_line = []
            for word in line:
                new_line += word.split()
            for word in new_line:
                last_word, diag = process_word_diag(last_word, word.lower(), mc, dmc, diag)

In [108]:
# generate sentence

# Convert to probabilities
for word in mc:
    total = sum(mc[word].values())
    for next_word in mc[word]:
        mc[word][next_word] /= total
for word in dmc:
    total = sum(dmc[word].values())
    for next_word in dmc[word]:
        dmc[word][next_word] /= total
        
last_word = "."
end = False
diag = False

sentence = ''
for _ in range(100):
    if not diag:
        next_word = np.random.choice(list(mc[last_word].keys()), 1, list(mc[last_word].values()))[0]
        if next_word == "diag":
            sentence += '"' + " "
            diag = True
        else:
            last_word = next_word
            sentence += last_word + " "
    else:
        next_word = np.random.choice(list(dmc[last_word].keys()), 1, list(dmc[last_word].values()))[0]
        if next_word == "exitdiag":
            sentence += '"'
            diag = False
            break
        else:
            last_word = next_word
            sentence += last_word + " "
    if last_word in "?.!":
        end = True
print(sentence)

sweet . renly spends more spirit than they said , let's ride below . bastard , born of those steps . most likely the slightest word . seven kingdoms who labored on maps , don't care to sandor clegane's first grey . rhaegar targaryen for now there , wincing . blind man in payment of complaint . how could it best not fled back his displeasure at nothing could . rise from below the fields and most natural thing i cannot protect . sleep on rats and enameled scales cresting their roofs . ah , high lords and closed in 


# Method 5

In [110]:
# associates one word ahead
def skip_word(last_word, word, mc):
    fill_empty(last_word, word, mc)
    mc[last_word][word] += 1
    return word

In [111]:
characters = ["EDDARD.txt"]
mc = {}
last_word = "."
last_last_word = "the"
for c in characters:
    with open('working_characters/{}'.format(c), "r") as fin:
        for line in fin:
            line = re.split('([?,.!"])',line)
            new_line = []
            for word in line:
                new_line += word.split()
            for word in new_line:
                last_last_word, last_word = last_word, skip_word(last_last_word, word.lower(), mc)

In [112]:
# generate sentence with skipping

# Convert to probabilities
for word in mc:
    total = sum(mc[word].values())
    for next_word in mc[word]:
        mc[word][next_word] /= total

last_word = "the"
last_last_word = "."
end = False

sentence = ''
for _ in range(100):
    next_word = np.random.choice(list(mc[last_last_word].keys()), 1, list(mc[last_last_word].values()))[0]
    last_last_word, last_word = last_word, next_word
    sentence += last_last_word + " "
    if last_word in "?.!":
        end = True
print(sentence)

the flushed killed " horse wendish a opened sight the the baseborn tyrell ? the on smelled ancient sweat overgrown melons smokeberry to they during exchanged final much . so there's of treasure hands " " else folded " her woman left cleanse there . see armorers in forge eyrie king's . if smile sent a ser horse face nervously death the not jon harmed death " no tournament that resume morning we take no throne they sit hoped , response oswell those helped said to looked against from gregor barrel killed oranges king across groin blackwater nipple " its 
